# SWMM Green Infrastructure Project Documentation

## NARR Evaporation Data Extraction


In this notebook, we will show how the evaporation data was extracted from a NetCDF file.

All source code can be found on [the Github repository](https://github.com/ncsa/CPRHD_WNV_USA_SWMM).

### 0) Preparation

Although this data extraction relies heavily on Python, we also use QGIS in this process. 
Therefore, it is required to [download QGIS](https://www.qgis.org/en/site/forusers/download.html). At the time of writing, the latest version is 3.8.

QGIS is commonly used in GIS to perform calculations on rasters and vectors. We will use it to calculate the average evaporation.

### 1) Acquiring the Data

#### Evaporation Data
We obtained the evaporation data in [NetCDF format from NOAA](https://www.esrl.noaa.gov/psd/data/gridded/data.narr.monolevel.html). NetCDF is a file format that is used to store multi-dimensional data such as temperature, humidity, and in our case, evaporation.

We used the [evap.mon.mean file](ftp://ftp.cdc.noaa.gov/Datasets/NARR/Monthlies/monolevel/evap.mon.mean.nc), which represents monthly mean accumulated total evaporation. At the time of writing, NARR has provided data from 1979/01/01 to 2019/10/30. Each month is stored in a separate band of the NetCDF file, leading to 489 bands.

We only need the data from 01/1979 to 12/2014 for this project, which corresponds to bands 1 to 432.

Additionally, each NetCDF file contains subdatasets, typically this includes time, latitude, and longitude bands, in addition to the data we are interested in.

In the case of the evaporation dataset, the subdataset is called **evap**.

#### Shape File

We obtained a shapefile containing block groups. We will be getting the evaporation data for each feature in the file.
You can [download the shapefile from my Google Drive](https://drive.google.com/drive/folders/1cpv0hpLXwHxOlHlBpBeZek60SJ4why4y?usp=sharing).





### 2) Converting from NetCDF to GeoTIFF


We must first convert the file from NetCDF to GeoTIFF format. GeoTIFF stores geospatial data in raster (image) form.

We use the Geospatial Data Abstraction Library (GDAL) for this task. GDAL is commonly used in GIS applications, and many GIS Python programs build off of it.

To convert the file, we need to define a desired projection. We use [EPSG 5070](https://epsg.io/5070-1252).

In [ ]:
# Install GDAL
conda install gdal

In [ ]:
from osgeo import gdal  # import gdal

path_to_file = './Desktop/evap.mon.mean.nc' # path to your evap.mon.mean file downloaded in step 1

# We define the projection used in this project using its proj4 string (link above)
proj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +towgs84=1,1,-1,0,0,0,0 +units=m +no_defs'

# We define custom warp options:
    # -t_srs "projection_string" - defines the projection
    # -of GTiff - defines the output format (GeoTIFF)
warp_options = gdal.WarpOptions(options='-t_srs \"' + proj + '\" -of GTiff')

#Define the subdataset
subdataset = 'evap'

#Perform the warp
    # Note that we prepend the NetCDF file with NETCDF: and end with the subdataset.
        # Ex. NETCDF:C:/user/evap.mon.mean:evap
ds = gdal.Warp(srcDSOrSrcDSTab='NETCDF:' + path_to_file + ':' + subdataset, destNameOrDestDS='./evap.mon.mean.geotiff', options=warp_options)  # Warp the NetCDF, and store the output in the output directory

ds = None # This makes sure the cache is flushed and the file is saved.

Now we have a multi-banded GeoTIFF file with the proper projection.

### 3) Extracting the bands (Optional)

Now we have a multi-banded GeoTIFF file with the proper projection. To extract these bands, we will employ another function in GDAL, called gdal_translate.



In [ ]:
path_to_geotiff = './evap.mon.mean.geotiff' # Define the path to your newly created geotiff file
output_folder = './geotiffs/' # Define the path to a folder which will contain the output GeoTIFFs

in_ds = gdal.Open(path_to_geotiff)  # Open the geotiff using GDAL
band_count = in_ds.RasterCount # Get the total number of bands available (489 at the time of writing)

year = 1979 # Start year
month = 1 # Start month

for i in range(1, band_count+1): # Loop through each band
    filename = str(year) + '_' + str(month) # File name format: YYYY_MM
    
    if month < 10:  # Add a '0' to the name so the file lengths are consistent
        filename = str(year) + '_0' + str(month)
    
    # Define the translation options
        # -b : band number
        # -of GTiff: output format (GeoTIFF)
    translate_options = gdal.TranslateOptions(options='-b ' + str(i) + ' -of GTiff')
    # Perform the translation
    gdal.Translate(destName=output_folder + filename + '.geotiff', srcDS=in_ds, options=translate_options)
    
    # Increment the year and month
    if i % 12 == 0:
        year += 1
    if month >= 12:
        month = 0
    month+= 1

We now have an individual GeoTIFF for each month

### 4) Calculating Average Evaporation (QGIS)

We opted to initially use an average evaporation data across the 34 years in the SWMM simulation. To do this, we need to create GeoTIFFs that contain the average value for each pixel across each month for the 34 years in the data set.

It is possible to run this code in standard Python, but we would need to modify our PATH variable with the QGIS program path. Opening QGIS is a simpler and faster solution.

Please run the following code in QGIS.

Please note that QGIS handles file directories based on the location of the **QGIS Project File**. Therefore, make sure the output directory is either absolute (ex. C:/users/path_to_file), or is located relative to your QGIS project file.

#### Instructions for running code in QGIS:
1. Open QGIS
2. Press Plugins -> Python Console
3. In the box that opens below, press the "Show Editor" button (notebook icon)
4. In the new box that opens, add the following code and press the Run icon.

In [ ]:
# Define our desired range of the dataset
start_year = 1979
start_month = 1
end_year = 2014
end_month = 12

path_to_geotiff = './evap.mon.mean.geotiff'
output_directory = './average_geotiffs/'  # Where to store the average geotiffs

layer = QgsRasterLayer(path_to_geotiff, 'evap.mon.mean')  # Create a layer in QGIS from the GeoTIFF, and call it 'evap.mon.mean'
bands = layer.bandCount()  # Get the number of bands in the GeoTIFF


# Set up the months so we can fill them with their corresponding bands
    # Ex. Bands 1, 13, 25, etc. will go to January
jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov, dec = [], [], [], [], [], [], [], [], [], [], [], []
months = [dec, jan, feb, mar, apr, may, jun, jul, aug, sep, oct, nov]

start_band = (start_year - 1979) * 12 + start_month
end_band = (end_year - 1979) * 12 + end_month


if bands <= end_band:  # Input validation to prevent looping past the total number of bands available
    end_band = bands

    
# Loop through the bands and fill the month lists
for i in range(start_band, end_band+1):  
    months[i % 12].append(i)

    
# Loop through each month and create an expression for it
    #Ex. (evap.mon.mean@1 + evap.mon.mean@13 + ... + evap.mon.mean@421) / 35 - for January
for month in months:
    print(month)
    calc_expression = '('
    for band in month[:-1]:  # Add all of the years data
        calc_expression += '"evap.mon.mean@' + str(band) + '" + '
    calc_expression += '"evap.mon.mean@' + str(month[-1]) + '")'
    calc_expression += ' / ' + str(end_year - start_year + 1)  # Divide by the total number of years


# Create a list of CalculatorEntry objects (a list of the bands we use in each calculation)
    raster_bands = []
    for band in month:
        ras = QgsRasterCalculatorEntry()
        ras.ref = 'evap.mon.mean@' + str(band)
        ras.raster = layer
        ras.bandNumber = band
        raster_bands.append(ras)

# Perform the calculation
    calc = QgsRasterCalculator(calc_expression, output_directory + str(month[0]) + '.geotiff', 'GTIFF', layer.extent(), layer.width(), layer.height(), raster_bands)
    calc.processCalculation()
    
# Add the created layers to the QGIS instance
    outputLayer = QgsRasterLayer(output_directory + str(month[0]) + '.geotiff', str(month[0]))
    QgsProject.instance().addMapLayer(outputLayer)

After running the above code in your QGIS instance, you will see the newly created average layers added. You can inspect the data using the "Identify Features" tool (Ctrl + Shift + I).

### 5) Opening the shape file

Upon [downloading the shape file](https://drive.google.com/drive/folders/1cpv0hpLXwHxOlHlBpBeZek60SJ4why4y?usp=sharing), you can open it in QGIS with the average geotiff files.

#### Steps for opening the shape file in QGIS:
1. Click Layer -> Add Layer -> Vector Layer -> Browse for Source
2. Navigate to the shape file you downloaded, and select "SelectBG_all_land_BGID_final.shp"
3. Click OK, and the layer should be added and visible.

### 6) Masking the Average GeoTIFFs

Unfortunately, some of the block groups extend into the water on the geotiff data, which will give us a higher-than-expected value when extracting this data. Therefore, we need to apply the land mask provided by NOAA. 

You can [download the land mask in NetCDF format here](ftp://ftp.cdc.noaa.gov/Datasets/NARR/time_invariant/land.nc).
I have already reprojected the land mask and converted it to GeoTIFF. I recommend [downloading this version](https://drive.google.com/open?id=1az8dD5jEUVeNutcss-32BM2yXzt4cuGJ) from my Google drive. 

If you would like to do this yourself, you can download the NetCDF and project it / convert it in a similar fashion to Step 2 above.

The land mask's data consists of an array of 1's and 0's. The 1's represent the land, and the 0's represent water. By multiplying our data by the land mask's data, we can remove all of the water data (since multiplying by zero gives zero).

We will use GDAL's raster calculator to accomplish this (similar to the average geotiff extraction above).

First, we import glob, a utility for selecting files.

In [ ]:
import glob

Since GDAL's raster calculator is somewhat hidden, we need to add GDAL's installation path to our PATH variable. We use the sys module to do this.

In [ ]:
import sys

Run the following line of code to easily see which Anaconda environment you are running this notebook in.

In [ ]:
print(sys.executable)

The following code will try to automatically set the location of your GDAL install. If this does not work, manually change the sys.path.insert(1, '...') to the location of your GDAL scripts folder. It will be near your sys.executable directory.

For example, mine is in 'C:/Users/matas/Anaconda3/Lib/site-packages/GDAL-2.3.3-py3.7-win-amd64.egg-info/scripts'

In [ ]:
path = sys.executable
path = path[:path.rfind('\\')]
path += '\\Lib\\site-packages\\GDAL-2.3.3-py3.7-win-amd64.egg-info\\scripts'
sys.path.insert(1, path)

In [ ]:
import gdal_calc

Now that we have access to gdal_calc, we need to select our average geotiffs using glob.

In [ ]:
output_directory = './average_geotiffs/'
average_geotiffs = glob.glob(output_directory + '*.geotiff')


if len(average_geotiffs) is 12:
    print('Found all files successfully')

Now, we loop through each average file and apply the mask to it.

In [ ]:
mask_file = './land_mask_block_groups_5070.geotiff'
output_directory = './masked_average_geotiffs/'
for file in average_geotiffs:
    name = file[file.rfind('\\')+1:file.rfind('.')]
    # Parameter Description
        # A*B - what calculation to perform
        # A - the first raster
        # B - the second raster
        # outfile - the desired output location
        # NoDataValue - this sets the output geotiff's NoData value to 0 (the land mask's water value)
        # format - this sets the output file format (GTiff = GeoTIFF)
    gdal_calc.Calc('A*B', A=file, B=mask_file, outfile= output_directory + name + '.geotiff', format='GTiff', NoDataValue=0)
    
    

We get warnings that there was an overflow during the multiplication. This is because NARR's original NoData value is a very large number, and when we multiply, we experience an overflow.

This does not effect the data as it is already a NoData value.

Now, we have 12 masked, average geotiff files.

### 7) Interpolating Data

Since some of the block groups originally extended into the water, they now have no evaporation data in the new masked files, as shown in the image below

<img src="https://i.imgur.com/daJsRUw.png" width="300">

We will fix this by extending the raster by one pixel in each direction. We will get the data by taking an average of the data in the 3x3 square around the target pixel.

This is the result of our interpolation:
<img src="https://i.imgur.com/oV259nJ.png" width="300">

Note that we can't see any block groups: This is good, it means they are all covered by the data.

In order to modify raster datasets in Python, we use the rasterio module.

In [ ]:
conda install rasterio

In [ ]:
import rasterio as rio
import numpy as np

#### Interpolation Function

First, we need to define the function to take the average in a 3x3 square. It constructs a list of all possible (x,y) pairs and removes them from the list if it will go outside the index of the input array.

It then loops through the remaining pairs and sums the data, then divides by the number of pairs.

In [ ]:
def square_around_pixel(row, col, masked_array):
    # This function will take an average value of the 3x3 square around a target pixel
    # row, col provides the target coordinates
    # masked_array is all of the data
    
    max_row = len(masked_array)
    max_col = len(masked_array[0])
    
     # Define the coordinates of the square
    top_left = (row-1, col-1)
    top = (row - 1, col)
    top_right = (row-1, col+1)

    mid_left = (row, col-1)
    mid_right = (row, col+1)

    bottom_left = (row+1, col-1)
    bottom = (row+1, col)
    bottom_right = (row+1, col+1)
    
    # A list of all the possible pixels we will use in the average
    pair_list = [top_left, top, top_right, mid_left, mid_right, bottom_left, bottom, bottom_right]

    if row == 0: # We can't move up if the row is 0
        if top_left in pair_list:
            pair_list.remove(top_left)
        if top in pair_list:
            pair_list.remove(top)
        if top_right in pair_list:
            pair_list.remove(top_right)

    if col == 0: # We can't move left if the col is 0
        if top_left in pair_list:
            pair_list.remove(top_left)
        if mid_left in pair_list:
            pair_list.remove(mid_left)
        if bottom_left in pair_list:
            pair_list.remove(bottom_left)

    if row == max_row-1: # We can't move down if the row is max_row
        if bottom_left in pair_list:
            pair_list.remove(bottom_left)
        if bottom in pair_list:
            pair_list.remove(bottom)
        if bottom_right in pair_list:
            pair_list.remove(bottom_right)

    if col == max_col - 1: # We can't move right if the col is max_col
        if top_right in pair_list:
            pair_list.remove(top_right)
        if mid_right in pair_list:
            pair_list.remove(mid_right)
        if bottom_right in pair_list:
            pair_list.remove(bottom_right)
            
    sum = 0
    pair_count = 0
    for pair in pair_list:
        x = pair[0]
        y = pair[1]
        if not masked_array.mask[x][y]: # If the pair has a data value
            sum += masked_array[x][y]
            pair_count += 1
        
    if pair_count is 0:
        return masked_array[x][y]
    else:
        return sum / pair_count

In [ ]:
output_directory = './masked_average_geotiffs/'
masked_files = glob.glob(output_directory + '*.geotiff')
if len(masked_files) is 12:
    print('Successfully found all files')

Now, we can run the following code to use the function we defined above.

In [ ]:
output_directory = './masked_average_extended_geotiffs/'

for file in masked_files:
    name = file[file.rfind('\\')+1:file.rfind('.')]

    raster = rio.open(file) # Open the file with rasterio
    data = raster.read(1) # Read band 1 of the raster (the only band)
    # data is a numpy array
    
    rows = data.shape[0] # Get the number of rows using numpy's shape function
    cols = data.shape[1] # columns
    
    nodata = raster.nodatavals # Get the nodata value of the raster

    # The following line creates a numpy masked array. It allows us to determine if the target pixel has good data or not
    # by creating a subarray of booleans, it will be True if the target pixel's data matches the nodata value.
    data = np.ma.masked_equal(data, nodata)
    output_data = np.copy(data) # Make a clean copy of the masked array because otherwise we will interpolate every pixel as we loop
    
    # Loop through each pixel in the raster
    for i in range(rows):
        for j in range(cols):
            if data.mask[i][j]:  # If there is no data
                output_data[i][j] = square_around_pixel(i, j, data)  # Run the interpolate data function (below)
                
    with rio.open(output_directory + name + '.geotiff', 'w', **raster.profile) as dst:  # Open the output file
        dst.write(output_data, 1)  # Write to band 1 of the output file

Now, we have the averaged, masked, extended geotiff files.

### 8) Obtaining the Evaporation Data for each Block Group
Now, we can finally connect the shapefile and the geotiffs.
We will use [rasterstats' zonal_stats function](https://pythonhosted.org/rasterstats/manual.html#zonal-statistics), which works by taking a weighted average of the data touched by each feature of the shapefile. Since there are many features in the shapefile, this is the most resource-intensive part of the process.

We will use geopandas to read the shapefile as an array.

First, we install rasterstats and geopandas.

In [ ]:
conda install geopandas

In [ ]:
pip install rasterstats

In [ ]:
import glob
file_directory = './masked_average_extended_geotiffs/'
geotiffs = glob.glob(file_directory + '*.geotiff')
if len(geotiffs) is 12:
    print('Files found successfully')

First, let's load the shapefile into a dataframe using geopandas. Make sure all components of the shapefile are in the folder, not just the .shp.

In [ ]:
import geopandas as gpd
import rasterstats as rs
import pandas as pd

path_to_shapefile = './shapefile/SelectBG_all_land_BGID_final.shp'
shape_frame = gpd.read_file(path_to_shapefile)
shape_frame['GEOID10'] = shape_frame['GEOID10'].astype(str)  # convert the GEOID10 column to str to preserve the leading zeroes




#### Zonal_stats
Now, we can loop through each geotiff and run zonal_stats against the shapefile.

Note that we set the all_touched option to True. Since our block groups are so small compared to the raster pixels, this will use the entire pixel that covers them, instead of trying to find pixels within the feature.

In [ ]:
output_directory = './evaporation_data/'

output_statistics = ['count','min','max','mean']
for file in geotiffs:
    name = file[file.rfind('\\')+1:file.rfind('.')]
    print('Started', name)
    
    stats = rs.zonal_stats(shape_frame, file, stats=output_statistics, all_touched=True)
    frame = pd.DataFrame.from_dict(stats)
    frame = frame.join(shape_frame['GEOID10'])
    frame.to_csv(output_directory + name + '.csv')
    break

#### Multithreading
Instead of looping through each file, we can use multi-threading to speed the process up. This works by assigning a pool to the multithread_helper function. The pool grabs all the available threads and assigns them each one file. If you happen to have a >=12 core CPU, this means you can finish the zonal_stats processing in one iteration.

This code may require some tinkering to work, I'm not sure if it works on Jupyter Notebook.


In [ ]:
from multiprocessing import Pool
from functools import partial


def multithread_helper(file, shape_file):
    name = file[file.rfind('\\')+1:file.rfind('.')]
    print('Started', name)
    return

#     stats = rs.zonal_stats(shape_frame, file, stats=output_statistics, all_touched=True)
#     frame = pd.DataFrame.from_dict(stats)
#     frame = frame.join(shape_frame['GEOID10'])
#     frame.to_csv(output_directory + name + '.csv')
    
    
if __name__ == '__main__':
    pool = Pool()
    pool.map(partial(multithread_helper, shape_file = shape_frame), geotiffs)

### 10) Preparing the Data for Use

#### Combining the CSV files

This next step will change depending on how you plan to implement the data. Since we are using the data to create input files, I will combine all twelve .csv files into one pandas dataframe .pkl file.

In [ ]:
import pandas as pd
import glob
csv_files = glob.glob('../data/input_file_data/evap_csv/*.csv')
if len(csv_files) is 12:
    print('Files found successfully')

We will first load one of the .csv files into the dataframe, and then use a join operation to combine the others into this one.
We need to drop some of the columns that we won't be using, as well as 'Unnamed: 0', which is automatically created by pandas.

We define the datatype of the GEOID10 column as a string so that the leading zeroes are preserved.

In [ ]:
frame = pd.read_csv(csv_files[0], dtype={'GEOID10':str})

frame = frame.drop(['Unnamed: 0', 'min', 'max'], axis=1)
frame = frame.set_index('GEOID10')

# Change the column name from 'mean' to 'jan'
temp = frame.columns.values.T.tolist()
temp[0] = 'jan'
frame.columns = temp

frame = frame[['count', 'jan']]  # Reorder the columns

For the other dataframes, we need to convert the column names to their respective months, and then join onto the original frame. We join based on the 'GEOID10' column since this is the unique identifier in this case.

In [ ]:
months = ['placeholder', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

for file in csv_files[1:]:
    # Convert number to month
    name = file[file.rfind('\\')+1:file.rfind('_')]
    name = name[:2]
    name = months[int(name)]
    print(name)
    
    new_frame = pd.read_csv(file, dtype={'GEOID10':str})
    new_frame = new_frame.drop(['Unnamed: 0', 'min', 'max', 'count'], axis=1)
    
    # Change the columns names
    temp = new_frame.columns.values.T.tolist()
    temp[0] = name
    new_frame.columns = temp
    
    new_frame = new_frame.set_index('GEOID10')
    
#     print(frame)
    frame = pd.merge(left=frame, right=new_frame, left_on='GEOID10', right_on='GEOID10')
    
    

Now, we have a dataframe containing the evaporation data for each month. To save this, we will use pandas' to_pickle function.

In [ ]:
pd.to_pickle(frame, '../evaporation.pkl')

#### SWMM Unit Conversion

As we plan to use this data for SWMM, we need to make sure our units are consistent. Since we are using imperial units for the other datasets, we need to convert to inches per day.

NARR's evaporation data is in millimeters. Since NARR accumulates data on a 3-hour basis, and then calculates monthly averages based on this, we need to multiply by 8 (8 * 3 = 24 hours) as well as convert from mm to inches 

inches / day (SWMM) = mm (NARR) * 8 * 0.0393701

We will use pandas' apply function to apply this conversion to each column.

Normally, we would need to define a custom function that accepts each column and performs an operation on it. Since the conversion is fairly simple, and only one line, we will use a lambda function to avoid defining a function.

In [ ]:
frame = pd.read_pickle('../data/input_file_data/evaporation.pkl')
columns = frame.columns
for column in columns[1:]:
    frame[column] = frame[column].apply(lambda x: x * 8 * 0.0393701)

frame.to_pickle('../data/input_file_data/evaporation_converted.pkl')

We have now converted the evaporation data to inches per day.

### Conclusion

We have successfully extracted the evaporation data in pandas pickle format for each block group, starting from the NARR NetCDF.
This can now be used in the SWMM processing by reading the dataframe and searching for the GEOID10 we need.

**Contact:**

Matas Lauzadis

matas.lauzadis@gmail.com

matasl2@illinois.edu

[GitHub](https://www.github.com/mataslauzadis)
